# 문서 벡터화 알고리즘 실습

220527

- 1) BOW
- 2) DTW
- 3) TF-IDF
- 4) 문서 검색 실습

In [1]:
# install
!python -m pip install konlpy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 97.3 MB/s 
     |████████████████████████████████| 453 kB 33.3 MB/s 


## 1) BoW (Bag of Words) 기반 문서 벡터화

- [ref](https://wikidocs.net/22650)

Bag of Words (단어들의 가방이)
- 단어들의 순서는 전혀 고려 없이 단어들의 출현 빈도(frequency)만으로 텍스트를 수치화
- 모든 텍스트 문서를 가방에 넣고 섞은 후 단어를 섞는다

BoW를 만드는 과정을 이렇게 두 가지 과정으로 생각해보겠습니다.

- 1) 각 단어에 고유한 정수 인덱스를 부여
- 2) 각 단어의 출현 빈도수 저장
- 불용어를 제거하면 더 좋은 성능

In [2]:
## bow function
from konlpy.tag import Okt
okt = Okt()

def build_bag_of_words(document):
    # 온점 제거 및 형태소 분석
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []
    for word in tokenized_document:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            # BoW에 전부 기본값 1을 넣는다.
            bow.insert(len(word_to_index) - 1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
            bow[index] = bow[index] + 1

    return word_to_index, bow

In [3]:
## 예시 1)
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)

print('input: %s'%(doc1))
print('\n1) 각 단어에 고유한 정수 인덱스를 부여\n   =>vocab:', vocab)
print('\n2) 각 단어의 출현 빈도수 저장\n   =>bag of words vector:', bow)

input: 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.

1) 각 단어에 고유한 정수 인덱스를 부여
   =>vocab: {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}

2) 각 단어의 출현 빈도수 저장
   =>bag of words vector: [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [4]:
## 예시 2)
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

vocab, bow = build_bag_of_words(doc2)

print('input: %s'%(doc2))
print('\n1) 각 단어에 고유한 정수 인덱스를 부여\n   =>vocab:', vocab)
print('\n2) 각 단어의 출현 빈도수 저장\n   =>bag of words vector:', bow)

input: 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.

1) 각 단어에 고유한 정수 인덱스를 부여
   =>vocab: {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}

2) 각 단어의 출현 빈도수 저장
   =>bag of words vector: [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [5]:
## 예시 3) 문서1과 문서2를 합쳐서 구할 수 있음
doc3 = doc1 + ' ' + doc2
vocab, bow = build_bag_of_words(doc3)

print('input: %s'%(doc3))
print('\n1) 각 단어에 고유한 정수 인덱스를 부여\n   =>vocab:', vocab)
print('\n2) 각 단어의 출현 빈도수 저장\n   =>bag of words vector:', bow)

input: 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.

1) 각 단어에 고유한 정수 인덱스를 부여
   =>vocab: {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}

2) 각 단어의 출현 빈도수 저장
   =>bag of words vector: [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


In [6]:
## 3. CountVectorizer 클래스로 BoW 만들기
# 단어의 빈도를 Count하여 Vector로 만드는 사이킷런의 CountVectorizer를 이용
from sklearn.feature_extraction.text import CountVectorizer

# corpus = ['you know I want your love. because I love you.']
corpus = [' '.join(okt.morphs(doc3))]

vector = CountVectorizer()

bow = vector.fit_transform(corpus).toarray()
vocab = sorted(vector.vocabulary_.items())

print('input: %s'%(doc3))
print('\n1) 각 단어에 고유한 정수 인덱스를 부여\n   =>vocab:', vocab)
print('\n2) 각 단어의 출현 빈도수 저장\n   =>bag of words vector:', bow)

input: 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.

1) 각 단어에 고유한 정수 인덱스를 부여
   =>vocab: [('기준', 0), ('느끼는', 1), ('느낀다', 2), ('다르다', 3), ('물가상승률', 4), ('발표', 5), ('상품', 6), ('소비', 7), ('소비자', 8), ('으로', 9), ('정부', 10), ('주로', 11), ('하는', 12)]

2) 각 단어의 출현 빈도수 저장
   =>bag of words vector: [[1 1 1 1 3 1 1 1 2 1 1 1 2]]


## 2) 문서 단어 행렬(Document-Term Matrix, DTM) 기반 문서 벡터화

- [ref](https://wikidocs.net/31698)

In [22]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'
doc3 = doc1 + ' ' + doc2

In [23]:
## DTW
from sklearn.feature_extraction.text import CountVectorizer

corpus = [doc1, doc2, doc3]
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
DTW = vector.fit_transform(corpus).toarray()
# 각 단어와 맵핑된 인덱스 출력
vocab = sorted(vector.vocabulary_.items())

print('input: %s'%(corpus))
print('\n1) 각 단어에 고유한 정수 인덱스를 부여\n   =>vocab:', vocab)
print('\n2) 각 단어의 출현 빈도수 저장\n   =>bag of words vector:\n', DTW)

input: ['정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.', '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.', '정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.']

1) 각 단어에 고유한 정수 인덱스를 부여
   =>vocab: [('기준으로', 0), ('느끼는', 1), ('느낀다', 2), ('다르다', 3), ('물가상승률과', 4), ('물가상승률은', 5), ('물가상승률을', 6), ('발표하는', 7), ('상품을', 8), ('소비자가', 9), ('소비자는', 10), ('소비하는', 11), ('정부가', 12), ('주로', 13)]

2) 각 단어의 출현 빈도수 저장
   =>bag of words vector:
 [[0 1 0 1 1 1 0 1 0 1 0 0 1 0]
 [1 0 1 0 0 0 1 0 1 0 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


## 3) TF-IDF 기반 문서 벡터화

- [ref](https://wikidocs.net/31698)


- (1) TF(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수
    - TF는 각 문서에서 각 단어의 등장 빈도
    
- (2) DF(t) : 특정 단어 t가 등장한 문서의 수

- (3) IDF(d, t) : df(t)에 반비례값

In [24]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

# corpus = [
#     'you know I want your love',
#     'I like you',
#     'what should I do ',    
# ]
corpus = [doc1, doc2, doc3]

tfidfv = TfidfVectorizer().fit(corpus)

TFIDF = tfidfv.transform(corpus).toarray()
vocab = sorted(tfidfv.vocabulary_.items())

print('input: %s'%(corpus))
print('\n1) 각 단어에 고유한 정수 인덱스를 부여\n   =>vocab:', vocab)
print('\n2) 각 단어의 TF-IDF\n   =>bag of words vector:\n', TFIDF)

input: ['정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.', '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.', '정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.']

1) 각 단어에 고유한 정수 인덱스를 부여
   =>vocab: [('기준으로', 0), ('느끼는', 1), ('느낀다', 2), ('다르다', 3), ('물가상승률과', 4), ('물가상승률은', 5), ('물가상승률을', 6), ('발표하는', 7), ('상품을', 8), ('소비자가', 9), ('소비자는', 10), ('소비하는', 11), ('정부가', 12), ('주로', 13)]

2) 각 단어의 TF-IDF
   =>bag of words vector:
 [[0.         0.37796447 0.         0.37796447 0.37796447 0.37796447
  0.         0.37796447 0.         0.37796447 0.         0.
  0.37796447 0.        ]
 [0.37796447 0.         0.37796447 0.         0.         0.
  0.37796447 0.         0.37796447 0.         0.37796447 0.37796447
  0.         0.37796447]
 [0.26726124 0.26726124 0.26726124 0.26726124 0.26726124 0.26726124
  0.26726124 0.26726124 0.26726124 0.26726124 0.26726124 0.26726124
  0.26726124 0.26726124]]


# 4) 벡터 유사도

- [url](https://wikidocs.net/24603)

In [10]:
# 4.1) 유클리드 거리(Euclidean distance)
import numpy as np

def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print('문서1과 문서Q의 거리 :',dist(doc1,docQ))
print('문서2과 문서Q의 거리 :',dist(doc2,docQ))
print('문서3과 문서Q의 거리 :',dist(doc3,docQ))

문서1과 문서Q의 거리 : 2.23606797749979
문서2과 문서Q의 거리 : 3.1622776601683795
문서3과 문서Q의 거리 : 2.449489742783178


In [11]:
# 4.2) 자카드 유사도(Jaccard similarity)
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print('문서1 :',tokenized_doc1)
print('문서2 :',tokenized_doc2)

union = set(tokenized_doc1).union(set(tokenized_doc2))
print('문서1과 문서2의 합집합 :',union)

intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print('문서1과 문서2의 교집합 :',intersection)

print('자카드 유사도 :',len(intersection)/len(union))

문서1 : ['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
문서2 : ['apple', 'banana', 'coupon', 'passport', 'love', 'you']
문서1과 문서2의 합집합 : {'likey', 'watch', 'holder', 'coupon', 'passport', 'love', 'like', 'apple', 'everyone', 'banana', 'card', 'you'}
문서1과 문서2의 교집합 : {'banana', 'apple'}
자카드 유사도 : 0.16666666666666666


In [12]:
## 4.3) 코사인 유사도(Cosine Similarity)
# 벡터의 방향(패턴)에 초점을 두므로 코사인 유사도는 문서의 길이가 다른 상황에서 비교적 공정한 비교
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

print('문서 1과 문서2의 유사도 :', cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :', cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :', cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


## 5) BM25, 유사도 기반 문서 검색

- [url](https://github.com/dorianbrown/rank_bm25)

In [13]:
## load, BM25 구현코드, https://github.com/dorianbrown/rank_bm25
# [비슷한 문장들의 idx], [비슷한 문장듯] = bm25_model.get_top_n(tokenized_query, question_list, n=5)
'''
- 사용법
corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)
query = "windy London"
tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)  # array([0.        , 0.93729472, 0.        ])
bm25.get_top_n(tokenized_query, corpus, n=1)  # 비슷한 문장들의 idx], [비슷한 문장듯]
'''

import math
import numpy as np
from multiprocessing import Pool, cpu_count
import pandas as pd

class BM25:
    def __init__(self, corpus, tokenizer=None):
        self.corpus_size = len(corpus)
        self.avgdl = 0
        self.doc_freqs = []
        self.idf = {}
        self.doc_len = []
        self.tokenizer = tokenizer

        if tokenizer:
            corpus = self._tokenize_corpus(corpus)

        nd = self._initialize(corpus)
        self._calc_idf(nd)

    def _initialize(self, corpus):
        nd = {}  # word -> number of documents with word
        num_doc = 0
        for document in corpus:
            self.doc_len.append(len(document))
            num_doc += len(document)

            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.doc_freqs.append(frequencies)

            for word, freq in frequencies.items():
                try:
                    nd[word]+=1
                except KeyError:
                    nd[word] = 1

        self.avgdl = num_doc / self.corpus_size
        return nd

    def _tokenize_corpus(self, corpus):
        pool = Pool(cpu_count())
        tokenized_corpus = pool.map(self.tokenizer, corpus)
        return tokenized_corpus

    def _calc_idf(self, nd):
        raise NotImplementedError()

    def get_scores(self, query):
        raise NotImplementedError()

    def get_batch_scores(self, query, doc_ids):
        raise NotImplementedError()

    def get_top_n(self, query, documents, n=5):

        # index도 출력되게 수정
        assert self.corpus_size == len(documents), "The documents given don't match the index corpus!"

        scores = self.get_scores(query)
        top_n = np.argsort(scores)[::-1][:n]
        return top_n.tolist(), [documents[i] for i in top_n]  # wygo 수정, idx도 출력되도록


class BM25Okapi(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        """
        Calculates frequencies of terms in documents and in corpus.
        This algorithm sets a floor on the idf values to eps * average_idf
        """
        # collect idf sum to calculate an average idf for epsilon value
        idf_sum = 0
        # collect words with negative idf to set them a special epsilon value.
        # idf can be negative if word is contained in more than half of documents
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_scores(self, query):
        """
        The ATIRE BM25 variant uses an idf function which uses a log(idf) score. To prevent negative idf scores,
        this algorithm also adds a floor to the idf value of epsilon.
        See [Trotman, A., X. Jia, M. Crane, Towards an Efficient and Effective Search Engine] for more info
        :param query:
        :return:
        """
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)
        score = np.zeros(len(doc_ids))
        doc_len = np.array(self.doc_len)[doc_ids]
        for q in query:
            q_freq = np.array([(self.doc_freqs[di].get(q) or 0) for di in doc_ids])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score.tolist()

    
def IR_BM25(query, bm25_model, tokenizer='space', n=5):
    '''
    input
        query: '. 윗글에서 확인할 수 있는 ㉠의 방법이 아닌 것은?'
        bm25_model: BM25Okapi(tokenized_corpus)
        tokenizer: mecab 형태소 분석기
    '''
    # 쿼리 프리프로세싱
#     query = '27. ㉠에 대한 이해로 가장 적절한 것은?'
#     query = '1. 윗글에서 확인할 수 있는 ㉠의 방법이 아닌 것은?'

    if tokenizer=='space':
        tokenized_query = query.split()
    else:
        tokenized_query = tokenizer(query)
    
    doc_representation = bm25_model.get_scores(tokenized_query)  # array([0.        , 0.93729472, 0.        ])

    # 유사문장 가져오기
    similar_idx, similar_sentences = bm25_model.get_top_n(tokenized_query, question_list, n)

    print('- query : %s' % (query))
    print('- similar\n%s' % ('\n'.join(similar_sentences)))
    
    return similar_idx, similar_sentences

In [14]:
# 0) DB에 있는 문제들을 가져와서 앞의 번호를 제거한 후 list를 만든다
question_list = ['8. (나)에 활용된 글쓰기 전략으로 적절하지 않은 것은?',
                 '9. <보기>는 (나)의 ‘학생’이 ‘초고’를 보완하기 위해 추가로 수집한 자료이다. 자료 활용 방안으로 적절하지 않은 것은?',
                 '10. 다음은 (나)의 ‘학생’이 ‘초고’를 고쳐 쓰는 과정에서 수행한 학습 활동이다. [A]에 들어갈 내용으로 가장 적절한 것은?',
                 '11. ㉠과 ㉡을 모두 충족하는 단어만을 <보기>에서 있는 대로 고른 것은?',
                 '12. 윗글과 <보기>를 바탕으로 추론한 내용으로 적절하지 않은 것은?',
                 '13. ⓐ～ⓔ는 잘못된 표기를 바르게 고친 것이다. 고치는 과정에서 해당 단어에 적용된 용언 활용의 예로 적절하지 않은 것은?',
                 '14. <학습 활동>을 수행한 결과로 적절하지 않은 것은? [3점]',
                 '15. <보기>의 ㉠과 ㉡에 들어갈 말로 적절한 것은?',
                 '16. (가), (나)에 대한 설명으로 가장 적절한 것은?',
                 '17. (가)의 ‘박제가’와 ‘이덕무’에 대한 이해로 적절하지 않은 것은?',
                 '18. 평등견 에 대한 이해로 가장 적절한 것은?',
                 '19. 문맥을 고려할 때 ㉠의 의미를 파악한 내용으로 가장 적절한 것은?',
                 '20. <보기>는 (가)에 제시된 \U000f0854북학의\U000f0855의 일부이다. [A]와 (나)를 참고하여 <보기>에 대해 비판적 읽기를 수행한 학생의 반응으로 적절하지 않은 것은? [3점]',
                 '21. 문맥상 ⓐ～ⓔ와 바꿔 쓰기에 가장 적절한 것은?',
                 '22. [A]와 [B]의 서술상 특징에 대한 설명으로 가장 적절한 것은?',
                 '23. 윗글에 대한 이해로 가장 적절한 것은?',
                 '24. ⓐ, ⓑ에 대한 이해로 적절하지 않은 것은?',
                 '25. <보기>를 참고하여 윗글을 감상한 내용으로 적절하지 않은 것은? [3점]',
                 '26. 윗글에 대한 이해로 적절하지 않은 것은?',
                 '27. ㉠에 대한 이해로 가장 적절한 것은?',
                 '28. 다음은 [A]에 제시된 예를 활용하여, 예약의 유형에 따라 예약상 권리자가 요구할 수 있는 급부에 대해 정리한 것이다. ㄱ～ㄷ에 들어갈 내용을 올바르게 짝지은 것은?',
                 '29. 윗글을 참고할 때, <보기>의 ㉮에 대한 이해로 적절하지 않은 것은? [3점]',
                 '30. 문맥상 ⓐ～ⓔ의 단어와 가장 가까운 의미로 쓰인 것은?',
                 '31. 윗글의 서술상 특징으로 가장 적절한 것은?',
                 '32. 윗글의 내용에 대한 이해로 적절하지 않은 것은?',
                 '33. <보기>를 참고하여 윗글을 감상한 내용으로 적절하지 않은 것은? [3점]',
                 '34. 윗글에 대한 이해로 적절하지 않은 것은?',
                 '35. 모델링 에 대한 설명으로 가장 적절한 것은?',
                 '36. ㉠에 대한 추론으로 적절한 것은?',
                 '37. 다음은 3D 애니메이션 제작을 위한 계획의 일부이다. 윗글을 바탕으로 할 때 적절하지 않은 것은? [3점]',
                 '38. (가)와 (나)에 대한 설명으로 가장 적절한 것은?',
                 '39. <보기>를 바탕으로 (가)를 감상한 내용으로 적절하지 않은 것은?',
                 '40. <보기>를 바탕으로 (나), (다)를 감상한 내용으로 적절하지 않은 것은? [3점]',
                 '41. (가)와 (다)를 비교하여 이해한 내용으로 가장 적절한 것은?',
                 '42. (다)에 대한 이해로 적절하지 않은 것은?',
                 '43. (가)에 대한 이해로 가장 적절한 것은?',
                 '44. ㉠～㉤의 의미를 고려하여 (나)를 감상한 내용으로 적절하지 않은 것은?',
                 '45. <보기>를 참고하여 (가)와 (나)를 이해한 내용으로 적절하지 않은 것은? [3점]',
                 '1. 윗글에서 확인할 수 있는 ㉠의 방법이 아닌 것은?',
                 '2. 윗글을 바탕으로 <보기>를 이해한 내용으로 적절하지 않은 것은? [3점]',
                 '3. 다음은 윗글을 읽은 학생의 반응이다. 이에 대한 설명으로 가장 적절한 것은?',
                 '4. 다음은 (가)와 (나)를 읽은 학생이 작성한 학습 활동지의 일부 이다. ㄱ～ㅁ에 들어갈 내용으로 적절하지 않은 것은?',
                 '5. 윗글에 대한 이해로 적절하지 않은 것은?',
                 '6. [A]에 대한 이해로 적절하지 않은 것은?',
                 '7. ㉠, ㉡에 대한 설명으로 가장 적절한 것은?',
                 '8. 는 윗글의 주제와 관련한 동서양 학자들의 견해이다. 윗글을 읽은 학생이 에 대해 보인 반응으로 적절하지 않은 것은? [3점]',
                 '9. ⓐ와 문맥상 의미가 가장 가까운 것은?',
                 '10. 윗글에서 베카리아의 관점으로 보기 어려운 것은?',
                 '11. ㉠에 대한 설명으로 적절하지 않은 것은?',
                 '12. 윗글을 바탕으로 베카리아의 입장을 추론한 내용으로 가장 적절한 것은? [3점]',
                 '13. 문맥상 ⓐ～ⓔ와 바꿔 쓰기에 적절하지 않은 것은?',
                 '14. 윗글에서 알 수 있는 내용으로 적절하지 않은 것은?',
                 '15. ㉠과 ㉡에 대한 설명으로 가장 적절한 것은?',
                 '16. 어느 바이러스 감염증의 진단 검사에 PCR를 이용하려고 한다. 윗글을 읽고 이해한 반응으로 가장 적절한 것은?',
                 '17. [A]를 바탕으로 의 실험 상황을 가정하고 와 같이 예상 결과를 추론하였다. ㉮～㉰에 들어갈 말로 적절한 것은?',
                 '18. [A]의 서술상 특징으로 가장 적절한 것은?',
                 '19. 서사의 흐름을 고려하여 ㉠～㉤에 대해 이해한 내용으로 적절 하지 않은 것은?',
                 '20. ⓐ，ⓑ에 대한 설명으로 가장 적절한 것은?',
                 '21. <보기>의 관점에서 윗글을 감상한 내용으로 적절하지 않은 것은? [3점]',
                 '22. (가)와 (나)의 공통점으로 가장 적절한 것은?',
                 '23. (나)에 대한 이해로 적절하지 않은 것은?',
                 '24. 문맥을 고려하여 ㉠～㉤에 대해 이해한 내용으로 적절하지 않은 것은?',
                 '25. (나)와 (다)를 비교하여 이해한 내용으로 가장 적절한 것은?',
                 '26. [A]와 [B]에 대한 이해로 가장 적절한 것은?',
                 '27. 를 바탕으로 (가)～(다)를 감상한 내용으로 적절하지 않은 것은? [3점]',
                 '28. 윗글의 내용에 대한 이해로 적절하지 않은 것은?',
                 '29. ⓐ와 ⓑ에 대한 설명으로 가장 적절한 것은?',
                 '30. [A]의 ‘달’에 대한 이해로 적절하지 않은 것은?',
                 '31. <보기>를 참고하여 ㉠～㉤을 이해한 내용으로 적절하지 않은 것은? [3점]',
                 '32. (가)와 (나)에 대한 설명으로 가장 적절한 것은?',
                 '33. (가), (나)의 시어에 대한 이해로 적절하지 않은 것은?',
                 '34. <보기>를 참고하여 (가), (나)를 감상한 내용으로 적절하지 않은 것은? [3점]',
                 '35. 위 강연자의 말하기 방식으로 가장 적절한 것은?',
                 '36. 다음은 동아리 부장이 강연자에게 보낸 전자 우편이다. 이를 바탕으로 세운 강연자의 계획 중 강연에 반영되지 않은 것은?',
                 '37. 다음은 학생이 강연을 들으면서 작성한 메모이다. 이를 바탕 으로 학생의 듣기 과정을 이해한 내용으로 적절하지 않은 것은? [3점]',
                 '38. 대화의 흐름을 고려할 때, ㉠～㉤에 대한 설명으로 적절하지 않은 것은?',
                 '39. [A]의 학생 1의 발화에 대한 설명으로 가장 적절한 것은?',
                 '40. (가)의 대화 내용이 (나), (다)에 각각 반영된 양상으로 적절 하지 않은 것은?',
                 '41. 작문 맥락을 고려할 때 (나), (다)에 대한 이해로 적절하지 않은 것은?',
                 '42. <보기>를 점검 기준으로 할 때 ⓐ, ⓑ를 고쳐 쓰기 위한 방안으로 가장 적절한 것은?',
                 '43. 다음은 초고를 작성하기 전에 학생이 떠올린 생각이다. ⓐ～ⓔ 중 학생의 초고에 반영되지 않은 것은?',
                 '44. 다음은 초고를 읽은 교지 편집부 담당 선생님의 조언이다. 이를 반영하여 [A]를 작성한 내용으로 가장 적절한 것은?',
                 '45. <보기>는 학생이 초고를 보완하기 위해 추가로 수집한 자료 이다. 자료의 활용 방안으로 적절하지 않은 것은? [3점]',
                 '35. ㉠과 ㉡을 모두 만족하는 용언의 짝으로 적절한 것은?',
                 '36. [A]를 바탕으로 의 ⓐ～ⓔ의 밑줄 친 부분을 이해한 내용으로 적절하지 않은 것은?',
                 '37. <학습 활동>을 수행한 결과로 적절한 것은? [3점]',
                 '38. <보기>의 ㉠～㉦에 대한 이해로 적절하지 않은 것은?',
                 '39. <보기>를 바탕으로 할 때, ㉠～㉢에 해당하는 단어가 사용된 예로 적절한 것은?',
                 '40. 위 화면을 통해 매체의 특성을 이해한 학생의 반응으로 가장 적절한 것은?',
                 '41. <보기>를 참고할 때, [A]에 대한 반응으로 적절하지 않은 것은? [3점]',
                 '42. 다음은 학생이 과제 수행을 위해 작성한 메모이다. 메모를 반영한 영상 제작 계획으로 적절하지 않은 것은?',
                 '43. (가), (나)에 대한 설명으로 가장 적절한 것은?',
                 '44. (가)의 언어적 특성을 고려할 때, ㉠～㉤에 대한 설명으로 적절하지 않은 것은?',
                 '1. 위 발표에 대한 설명으로 가장 적절한 것은?',
                 '2. 다음은 발표를 하기 위해 작성한 메모와 발표 계획이다. 발표 내용에 반영되지 않은 것은?',
                 '3. <보기>는 위 발표를 들은 학생들의 반응이다. 발표의 내용을 고려하여 학생의 반응을 이해한 내용으로 가장 적절한 것은?',
                 '4. (가)에 나타난 의사소통 방식으로 적절하지 않은 것은?',
                 '5. <보기1>은 ‘지도사’가 받은 전자 우편의 내용이고, 는 ‘지도사’가 인터뷰를 위해 준비한 자료이다. ㉠～㉢의 활용 계획 중 (가)에 드러나지 않은 것은? [3점]',
                 '6. (가)와 (나)를 고려할 때, 학생이 글을 쓰기 위해 떠올렸을 생각으로 적절하지 않은 것은?',
                 '7. 다음을 고려할 때, [A]에 들어갈 내용으로 가장 적절한 것은?',
                 '8. ㉠～㉤ 중 (나)에 반영되지 않은 것은?',
                 '9. <보기>는 [A]의 초고이다. 를 [A]로 고쳐 쓸 때 반영한 친구의 조언으로 가장 적절한 것은?',
                 '10. 다음은 (나)를 읽은 학생이 이를 참고하여 작성한 글의 일부 이다. (나)의 정보를 활용한 방식으로 가장 적절한 것은? [3점]',
                 '11. <보기>의 ㉮에 들어갈 말로 적절한 것은?',
                 '12. 윗글을 읽고 추론한 내용으로 적절하지 않은 것은?',
                 '13. <보기>의 [자료]에서 ㉠에 해당하는 단어만을 있는 대로 고른 것은? [3점]',
                 '14. <학습 활동>을 수행한 결과로 적절한 것은?',
                 '15. <보기>에 대한 이해로 적절한 것은?',
                 '16. [A]의 서술상 특징에 대한 설명으로 가장 적절한 것은?',
                 '17. [B]에 대한 이해로 적절하지 않은 것은?',
                 '18. 요구 조건 을 중심으로 윗글을 이해한 내용으로 적절하지 않은 것은?',
                 '19. <보기>를 참고하여 윗글을 감상한 내용으로 적절하지 않은 것은? [3점]',
                 '20. (가)와 (나)의 공통적인 내용 전개 방식으로 가장 적절한 것은?',
                 '21. (가)의 형식론 에 대한 이해로 가장 적절한 것은?',
                 '22. (가)에 등장하는 이론가와 예술가들이 상대의 견해나 작품을 평가할 수 있는 말로 적절하지 않은 것은?',
                 '23. 다음은 비평문을 쓰기 위해 미술 전람회에 다녀온 학생이 (가)와 (나)를 읽은 후 작성한 메모의 일부이다. 메모의 내용이 적절하지 않은 것은? [3점]',
                 '24. 피카소의 게르니카 에 대해 <보기>의 A는 ㉠의 관점, B는 ㉡의 관점에서 비평한 내용이다. (나)를 바탕으로 A, B를 이해한 내용으로 적절하지 않은 것은?',
                 '25. 문맥을 고려할 때, 밑줄 친 말이 ⓐ～ⓔ의 동음이의어인 것은?',
                 '26. 윗글의 내용과 일치하는 것은?',
                 '27. ㉠의 이유로 가장 적절한 것은?',
                 '28. 행정규칙 에 관한 설명 중 적절하지 않은 것은?',
                 '29. 윗글을 바탕으로 <보기>의 ㉮～㉰에 대해 이해한 내용으로 가장 적절한 것은? [3점]',
                 '30. 문맥상 ⓐ～ⓔ와 바꿔 쓰기에 가장 적절한 것은?',
                 '31. ㉠에 대한 이해로 적절하지 않은 것은?',
                 '32. [A]에 대한 설명으로 가장 적절한 것은?',
                 '33. <보기>를 참고하여 윗글을 감상한 내용으로 적절하지 않은 것은? [3점]',
                 '34. 윗글에서 답을 찾을 수 있는 질문에 해당하지 않는 것은?',
                 '35. 윗글을 읽고 이해한 내용으로 적절하지 않은 것은?',
                 '36. ㉠～㉢에 대한 설명으로 적절한 것은?',
                 '37. <보기>는 윗글을 읽은 학생이 ‘가상의 실험 결과’를 보고 추론한 내용이다. [가]에 들어갈 말로 적절하지 않은 것은? [3점]',
                 '38. (나)의 시상 전개에 대한 설명으로 가장 적절한 것은?',
                 '39. (가)를 참고하여 (나)를 감상한 내용으로 적절하지 않은 것은?',
                 '40. (다)를 이해한 내용으로 적절하지 않은 것은?',
                 '41. ㉠, ㉡에 대한 설명으로 가장 적절한 것은?',
                 '42. ⓐ를 바탕으로 (나), (다)를 이해한 내용으로 적절하지 않은 것은? [3점]',
                 '43. (가)에 대한 이해로 가장 적절한 것은?',
                 '44. ㉠～㉤에 대한 이해로 적절하지 않은 것은?',
                 '45. <보기>를 참고하여 (가), (나)를 감상한 내용으로 적절하지않은 것은? [3점]']

print(question_list[-1])
# 문제에서 번호 제거
# '. ' 기준으로 문제 제거, ex) '37. '를 없앰
question_list = [' '.join(tmp.split('. ')[1:]) for tmp in question_list]

print('번호 제거 ==>> ', question_list[-1])

45. <보기>를 참고하여 (가), (나)를 감상한 내용으로 적절하지않은 것은? [3점]
번호 제거 ==>>  <보기>를 참고하여 (가), (나)를 감상한 내용으로 적절하지않은 것은? [3점]


In [15]:
## 1) DB에 있는 문장들을 형태소 단위로 쪼개준다
tokenizer = 'okt'  # konlpy, no need java, only python

# load question list
sentence = '37. 다음은 3D 애니메이션 제작을 위한 계획의 일부이다. 윗글을 바탕으로 할 때 적절하지 않은 것은? [3점]'

from konlpy.tag import Okt
tokenizer = Okt().morphs
print('okt check :', tokenizer(sentence))
tokenized_corpus = [tokenizer(sentence) for sentence in question_list]  # 형태소 기반 분절


okt check : ['37', '.', '다음', '은', '3', 'D', '애니메이션', '제작', '을', '위', '한', '계획', '의', '일부', '이다', '.', '윗', '글', '을', '바탕', '으로', '할', '때', '적절하지', '않은', '것', '은', '?', '[', '3', '점', ']']


In [16]:
## 2) 모든 문제들의 BM25 벡터를 미리 구한다
# BM25 계산
bm25_model = BM25Okapi(tokenized_corpus)
print('\n',tokenized_corpus[4])


 ['윗', '글', '과', '<', '보기', '>', '를', '바탕', '으로', '추론', '한', '내용', '으로', '적절하지', '않은', '것', '은', '?']


In [17]:
query = '37. 다음은 3D 애니메이션 제작을 위한 계획의 일부이다. 윗글을 바탕으로 할 때 적절하지 않은 것은? [3점]'
similar_idx, similar_sentences = IR_BM25(query, bm25_model, tokenizer, n=5)

- query : 37. 다음은 3D 애니메이션 제작을 위한 계획의 일부이다. 윗글을 바탕으로 할 때 적절하지 않은 것은? [3점]
- similar
다음은 3D 애니메이션 제작을 위한 계획의 일부이다 윗글을 바탕으로 할 때 적절하지 않은 것은? [3점]
다음은 학생이 과제 수행을 위해 작성한 메모이다 메모를 반영한 영상 제작 계획으로 적절하지 않은 것은?
윗글을 바탕으로 <보기>를 이해한 내용으로 적절하지 않은 것은? [3점]
다음은 학생이 강연을 들으면서 작성한 메모이다 이를 바탕 으로 학생의 듣기 과정을 이해한 내용으로 적절하지 않은 것은? [3점]
윗글을 바탕으로 베카리아의 입장을 추론한 내용으로 가장 적절한 것은? [3점]


In [18]:
query = '39. <보기>를 바탕으로 할 때, ㉠～㉢에 해당하는 단어가 사용된 예로 적절한 것은?'
similar_idx, similar_sentences = IR_BM25(query, bm25_model, tokenizer)

- query : 39. <보기>를 바탕으로 할 때, ㉠～㉢에 해당하는 단어가 사용된 예로 적절한 것은?
- similar
<보기>를 바탕으로 할 때, ㉠～㉢에 해당하는 단어가 사용된 예로 적절한 것은?
ⓐ～ⓔ는 잘못된 표기를 바르게 고친 것이다 고치는 과정에서 해당 단어에 적용된 용언 활용의 예로 적절하지 않은 것은?
<보기>의 [자료]에서 ㉠에 해당하는 단어만을 있는 대로 고른 것은? [3점]
<보기>를 점검 기준으로 할 때 ⓐ, ⓑ를 고쳐 쓰기 위한 방안으로 가장 적절한 것은?
<보기>를 참고할 때, [A]에 대한 반응으로 적절하지 않은 것은? [3점]


In [19]:
query = '27. ㉠에 대한 이해로 가장 적절한 것은?'
similar_idx, similar_sentences = IR_BM25(query, bm25_model, tokenizer)

- query : 27. ㉠에 대한 이해로 가장 적절한 것은?
- similar
㉠에 대한 이해로 가장 적절한 것은?
㉠에 대한 이해로 적절하지 않은 것은?
㉠의 이유로 가장 적절한 것은?
㉠, ㉡에 대한 설명으로 가장 적절한 것은?
㉠, ㉡에 대한 설명으로 가장 적절한 것은?


In [20]:
query = '25. 문맥을 고려할 때, 밑줄 친 말이 ⓐ～ⓔ의 동음이의어인 것은?'
similar_idx, similar_sentences = IR_BM25(query, bm25_model, tokenizer)

- query : 25. 문맥을 고려할 때, 밑줄 친 말이 ⓐ～ⓔ의 동음이의어인 것은?
- similar
문맥을 고려할 때, 밑줄 친 말이 ⓐ～ⓔ의 동음이의어인 것은?
[A]를 바탕으로 의 ⓐ～ⓔ의 밑줄 친 부분을 이해한 내용으로 적절하지 않은 것은?
문맥을 고려할 때 ㉠의 의미를 파악한 내용으로 가장 적절한 것은?
대화의 흐름을 고려할 때, ㉠～㉤에 대한 설명으로 적절하지 않은 것은?
다음을 고려할 때, [A]에 들어갈 내용으로 가장 적절한 것은?


## 문서 분석 완료!